<a href="https://colab.research.google.com/github/gkbichara/Match-Predictor/blob/main/Data_Scraping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
# narrow down the HTML to that of just the table
standings_table = soup.select('table.stats_table')[0]

In [7]:
# find all the links to the pages of the individual teams
# use find_all instead of select to just find the tags, can use either one
links = standings_table.find_all('a')

In [8]:
# goes through each of the a elements and finds the href values
# finds all the hyperlinks that are in the original table on fbref
links = [l.get("href") for l in links]

In [9]:
# from the hyperlinks in the table, this filters them further to only be squad links
links = [l for l in links if '/squads/' in l]

In [10]:
# links only had the latter part of the link, adding the domains to make them full URLS
# will do this using Python string formatting
team_urls = [f"https://fbref.com{l}" for l in links]

In [11]:
# using the first link to try scraping data
team_url = team_urls[0]

In [12]:
# use requests to pull the HTML from the website
data = requests.get(team_url)

In [13]:
import pandas as pd

# match looks for a string in a table
# pd.read_html scans all the table tags and we look for the one with the string in the match
matches = pd.read_html(data.text, match = "Scores & Fixtures")

<ipython-input-13-26acb6019267>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match = "Scores & Fixtures")


In [14]:
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,NaN,NaN,56.0,78146.0,Rúben Dias,4-3-3,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,0.8,1.0,52.0,39818.0,Kevin De Bruyne,3-2-4-1,4-2-3-1,NaN,Match Report,NaN
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,3.3,0.3,75.0,NaN,Kevin De Bruyne,3-2-4-1,5-4-1,NaN,Match Report,NaN
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,3.0,0.7,67.0,NaN,Kevin De Bruyne,3-2-4-1,4-2-3-1,Michael Oliver,Match Report,NaN
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,NaN,NaN,NaN,Brentford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN


In [15]:
# initialize another instance of BeautifulSoup
soup = BeautifulSoup(data.text)

In [16]:
# find all links on the page
links = soup.find_all('a')

In [17]:
# use list comprehension to get the URL's
links = [l.get("href") for l in links]

In [18]:
# use list comprehension to filter further and find shooting stats
# 'all_comps/shooting/' adding this to filter for shooting stats
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [19]:
# the 4-time repetition just shows us that the link is on the page 4 times
links

['/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [20]:
# download HTML of the shooting stats page
# using a formatted string to add the initial part of the URL, to make it complete
data = requests.get(f"https://fbref.com{links[0]}")

In [21]:
# filter out the shooting part of the table
# using pandas to make it a much simpler process
shooting = pd.read_html(data.text, match = "Shooting")[0]

<ipython-input-21-bad1746dc796>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match = "Shooting")[0]


In [22]:
shooting.head()

For Manchester City                                                        \
                 Date   Time                 Comp                Round  Day   
0          2024-08-10  15:00  FA Community Shield  FA Community Shield  Sat   
1          2024-08-18  16:30       Premier League          Matchweek 1  Sun   
2          2024-08-24  15:00       Premier League          Matchweek 2  Sat   
3          2024-08-31  17:30       Premier League          Matchweek 3  Sat   
4                 NaN    NaN                  NaN                  NaN  NaN   

                                              ... Standard                \
  Venue Result     GF     GA        Opponent  ...     Dist   FK PK PKatt   
0  Home      D  1 (7)  1 (6)  Manchester Utd  ...      NaN  NaN  0     0   
1  Away      W      2      0         Chelsea  ...     19.1  0.0  0     0   
2  Home      W      4      1    Ipswich Town  ...     17.8  1.0  1     1   
3  Away      W      3      1        West Ham  ...     15.0  1.0  0     0   
4   NaN     --    NaN    NaN             NaN  ...     16.7  2.0  1     1   

  Expected                            Unnamed: 25_level_0  
        xG npxG npxG/Sh  G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN   NaN     NaN        Match Report  
1      0.8  0.8    0.07   1.2     1.2        Match Report  
2      3.3  2.6    0.20   0.7     0.4        Match Report  
3      3.0  3.0    0.13   0.0     0.0        Match Report  
4      NaN  NaN    0.13  10.0     9.0                 NaN  

[5 rows x 26 columns]

In [23]:
# dropping an index level as we do not need multi-index levels, so we will be dropping
shooting.columns = shooting.columns.droplevel()

In [24]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,19.1,0.0,0,0,0.8,0.8,0.07,1.2,1.2,Match Report
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,17.8,1.0,1,1,3.3,2.6,0.20,0.7,0.4,Match Report
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,15.0,1.0,0,0,3.0,3.0,0.13,0.0,0.0,Match Report
4,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,...,16.7,2.0,1,1,NaN,NaN,0.13,10.0,9.0,NaN


In [25]:
# merging shooting data to match data
# only merging a few distinct columns
# merging on date to ensure it's the same game
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")

In [26]:
# using a loop we will be scraping multiple years' worth of data
years = list(range(2025,2020, -1))

In [27]:
# creating dataframes, one for each team each season
# will combine into one big dataframe in the end
all_matches = []

In [28]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [29]:
"""
getting the HTML from each season's standings
using beautifulSoup to parse HTML
looking for the stats table for match data
find all the url's
filter to team links
turn them from relative to absolute links

then go through each team to make sure it has its own match log
split url to get name using / to split, and -1 as index as name is at the end
replace stats with nothing to have the name

parsing scores and fixtures table
pulling the all comps shooting link, which gets the shooting stats
converting it to an absolute URL
use pandas to read in shooting stats
dropping top index level
merge shooting stats with match stats
use a try and except in case team does not have shooting stats

filter comp so that it only contains prem
adding extra columns because website does not have this, as it assumes you know them
add to all matches list
sleep at the end to avoid getting blocked
"""
!pip install beautifulsoup4
!pip install pandas
!pip install requests
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

for year in years:
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]

    # check if there are any links
    if len(links) > 0:
      data = requests.get(f"https://fbref.com{links[0]}")

      # check if request was successful
      if data.status_code == 200:
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
          team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
          continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
    # increase sleep time to avoid overloading the website with requests
    time.sleep(random.uniform(10, 20))

<ipython-input-29-9596075d2162>:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]
<ipython-input-29-9596075d2162>:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match = "Shooting")[0]
<ipython-input-29-9596075d2162>:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]
<ipython-input-29-9596075d2162>:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'St

In [31]:
match_df = pd.concat(all_matches)

In [32]:
match_df.columns = [c.lower() for c in match_df.columns]

In [33]:
match_df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Match Report,NaN,11.0,5.0,19.1,0.0,0,0,2025,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Match Report,NaN,13.0,4.0,17.8,1.0,1,1,2025,Manchester City
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,23.0,8.0,15.0,1.0,0,0,2025,Manchester City
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Everton,...,Match Report,NaN,10.0,5.0,13.8,0.0,0,0,2025,Brighton and Hove Albion
1,2024-08-24,12:30,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,Manchester Utd,...,Match Report,NaN,14.0,4.0,14.2,1.0,0,0,2025,Brighton and Hove Albion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,18.2,0.0,0,0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,13.4,1.0,0,0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,18.5,0.0,0,0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,18.3,1.0,0,0,2021,Sheffield United


In [34]:
match_df.to_csv("matches.csv")